# CISB5123 Text Analytics
## Lab Assignment 1

Group pair:
1. Name: Muhammad Ilham Bin Muhammad Redzuan Phua (SW01081025)
2. Name: Muhammad Hafeez Bin Muhammad Ramizal (SW01081025)

In [19]:
# Import all required libraries
import requests
import csv
import json
import re
import pandas as pd
from datetime import datetime

In [20]:
# Define the product URL
url = 'https://shopee.com.my/Logitech-G-PRO-PRO-X-Superlight-Wireless-Gaming-Mouse-i.49025587.3360835555?sp_atk=2272c440-13e8-4491-a326-4f31e8b348ca&xptdk=2272c440-13e8-4491-a326-4f31e8b348ca'

# Function to look for specific pattern in the URL
r = re.search(r'i\.(\d+)\.(\d+)', url)

# If the pattern is found, extract the data
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()



In [21]:
# Initialize an empty list to store the review
review_list = []

In [22]:
# Initialize offset to 0 which used to paginate through the reviews
offset = 0

# Initialize a counter for the number of pages
page_count = 1

# Enters while loop to fetch reviews in batches of 20 (as indicated by the 'limit=20' query parameter in the request URL)
while True:
    ratings_url = f'https://shopee.com.my/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    data = requests.get(ratings_url).json()

    # Iterate through each rating and append it to the data list
    for rating in data['data']['ratings']:
        author_username = rating['author_username']

        # Convert Unix timestamp to readable date and time
        timestamp = rating['ctime']
        readable_time = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        comment = rating['comment']

        # Append a dictionary for each rating to the review_list
        review_list.append({
            'Reviewer Username': author_username,
            'Date of review': readable_time,
            'Comment': comment
        })

    # Check if the number of reviews fetched less than 20,
    # or page_count already reach 5 pages. If so, then break
    # the loop
    if len(data['data']['ratings']) < 20 or page_count >= 5:
        break

    # Increase the page count
    page_count += 1

    # Increase offset by 20 to fetch the next page
    offset += 20

In [24]:
# Save the review to a CSV file
with open('shopee.csv', 'w', encoding='utf-8', newline='') as csvfile:
    fieldnames = ['Reviewer Username', 'Date of review', 'Comment']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write headers
    writer.writeheader()

    # Write quotes
    for review in review_list:
        writer.writerow(review)

# Other method to convert review_list to CSV file
# using pandas DataFrame
df = pd.DataFrame(review_list)

# Save the DataFrame to a CSV file
df.to_csv('shopee.csv', index=False)

# Once done print status message
print("All reviews have been saved to shopee.csv")

All reviews have been saved to shopee.csv
